In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report
import unicodedata
import pythainlp
import re
import emoji

In [2]:
def normalize_double_quote(text: str):
    all_quote = [
    '“', # U+201c
    '”', # U+201d
    "„", # U+201e
    '«', # U+00AB
    '»', # U+00BB
    '„', # U+201E
    '“', # U+201C
    '‟', # U+201F
    '”', # U+201D
    '❝', # U+275D
    '❞', # U+275E
    '〝', # U+301D
    '〞', # U+301E
    '〟', # U+301F
    '＂', # U+FF02
    ]
    std_quote = "\"" #U+0022
    table = str.maketrans(dict.fromkeys(all_quote, std_quote))
    return text.translate(table)

def normalize_single_quote(text: str):
    all_quote = ['\u02BB', '\u02BC', '\u066C', '\u2018', '\u2019', '\u201A', '\u275B', '\u275C']
    std_quote = "\'"
    table = str.maketrans(dict.fromkeys(all_quote, std_quote))
    return text.translate(table)

def replace_url(text):
    URL_PATTERN = r"""(?i)\b((?:https?:(?:/{1,3}|[a-z0-9%])|[a-z0-9.\-]+[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)/)(?:[^\s()<>{}
]+|
[^\s()]*?\)|
)+(?:
[^\s()]*?\)|
|[^\s`!()
{};:'".,<>?«»“”‘’])|(?:(?<!@)[a-z0-9]+(?:[.\-][a-z0-9]+)*[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)\b/?(?!@)))"""
    return re.sub(URL_PATTERN, 'xxurl', text)

def replace_rep(text):
    def _replace_rep(m):
        c,cc = m.groups()
        return f'{c}xxrep'
    re_rep = re.compile(r'(\S)(\1{2,})')
    return re_rep.sub(_replace_rep, text)

def ungroup_emoji(toks):
    res = []
    for tok in toks:
        if emoji.emoji_count(tok) == len(tok):
            for char in tok:
                res.append(char)
        else:
            res.append(tok)
    return res

def preprocess_func(text):
    new_text = unicodedata.normalize('NFKD', text)
    new_text = pythainlp.util.normalize(new_text)
    new_text = normalize_double_quote(new_text)
    new_text = normalize_single_quote(new_text)

    new_text = new_text.lower().strip()
    new_text = replace_url(new_text)
    new_text = replace_rep(new_text)

    res = [word for word in pythainlp.word_tokenize(new_text) if word and not re.search(pattern=r"\s+", string=word)]

    res = ungroup_emoji(res)
    return res

In [3]:
train_df = pd.read_csv('./train_data_fake_news.csv')
train_back_df = pd.read_csv('./train_augment_data.csv')
val_df = pd.read_csv('./valid_data_fake_news.csv')
test_df = pd.read_csv('./test_data_fake_news.csv')

In [4]:
train_back_df

,label,text
0,0,กระทรวงสาธารณสุขจัดประชุมอย่างเร่งด่วน! ไวรัส ...
1,1,อย่าทิ้ง! เปลือกมะนาวสามารถบรรเทาอาการปวดข้อ |...
2,0,หิมะฝรั่งเศส! เร่งถนนบนภูเขาหลังจากนักท่องเที่...
3,0,ผู้เชี่ยวชาญชี้ให้เห็นว่าผู้คนแห่กันซื้อเนื้อเ...
4,1,พบโครงกระดูกเก่าที่ถ้ำใต้น้ำ เม็กซิโกมีอายุมาก...
...,...,...
1899,0,ต้องเรียนรู้ที่จะกิน! เม็ดมะม่วงหิมพานต์กินกิน...
1900,1,5 ขั้นตอนง่ายๆในการนวดขนาดหน้าอกให้ใหญ่ขึ้นด้ว...
1901,1,การแช่ในผ้าด้วยผงซักฟอกที่เข้มข้นมากที่เราเคยท...
1902,1,การเปิดเผยสูตรฟันขาว 5 สูตรที่จะทำให้ฟันของคุณ...


In [5]:
train_val_df = pd.concat([train_df, val_df, train_back_df]).reset_index().drop(columns='index')

In [6]:
train_val_df

,text,label
0,ไก่ลุงเคนกินแล้วมีแต่สารเร่งโต,1
1,เครื่องดื่มต้านCovid-19ค่าพีเอชคือความเป็นด่าง...,1
2,"แค่ฟังชื่อก็ขนลุก!""ต้นป่าช้าเหงา""สมุนไพรชั้นดี...",1
3,พิชัย'แนะ'ประยุทธ์'เพิ่มเซลล์สมองแก้ปัญหาเศรษฐ...,0
4,อุจจาระตกค้างเรื่องใหญ่!แจกสูตรดีท็อกซ์ล้างลำไ...,1
...,...,...
4768,ต้องเรียนรู้ที่จะกิน! เม็ดมะม่วงหิมพานต์กินกิน...,0
4769,5 ขั้นตอนง่ายๆในการนวดขนาดหน้าอกให้ใหญ่ขึ้นด้ว...,1
4770,การแช่ในผ้าด้วยผงซักฟอกที่เข้มข้นมากที่เราเคยท...,1
4771,การเปิดเผยสูตรฟันขาว 5 สูตรที่จะทำให้ฟันของคุณ...,1


In [27]:
tfidf = TfidfVectorizer(tokenizer=preprocess_func, ngram_range=(1,2), min_df=20, sublinear_tf=True)
tfidf_fit = tfidf.fit(train_val_df['text'])
text_train = tfidf_fit.transform(train_val_df['text'])
text_test = tfidf_fit.transform(test_df['text'])

c:\Users\foofo\Desktop\workspace\fake_news_project\venv\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [28]:
text_train.shape, text_test.shape

((4773, 14703), (507, 14703))

In [11]:
X_train = text_train.toarray()
X_test = text_test.toarray()
X_train.shape, X_test.shape

((4773, 14703), (507, 14703))

In [29]:
y_train = train_val_df['label']
y_test = test_df['label']

In [30]:
model = KNeighborsClassifier(n_neighbors=5)
model.fit(X_train, y_train)
model.score(X_test, y_test)

0.9072978303747534

In [31]:
from sklearn.metrics import classification_report
y_val_pred = model.predict(X_test)
print(classification_report(y_test, y_val_pred))

              precision    recall  f1-score   support

           0       0.89      0.89      0.89       212
           1       0.92      0.92      0.92       295

    accuracy                           0.91       507
   macro avg       0.90      0.91      0.90       507
weighted avg       0.91      0.91      0.91       507



In [15]:
model_2 = LogisticRegression()
model_2.fit(X_train, y_train)
model_2.score(X_test, y_test)

0.9250493096646942

In [16]:
y_val_pred_2 = model_2.predict(X_test)
print(classification_report(y_test, y_val_pred_2))

              precision    recall  f1-score   support

           0       0.92      0.90      0.91       212
           1       0.93      0.95      0.94       295

    accuracy                           0.93       507
   macro avg       0.92      0.92      0.92       507
weighted avg       0.92      0.93      0.92       507



In [17]:
model_3 = GaussianNB()
model_3.fit(X_train, y_train)
model_3.score(X_test, y_test)

0.883629191321499

In [18]:
y_val_pred_3 = model_3.predict(X_test)
print(classification_report(y_test, y_val_pred_3))

              precision    recall  f1-score   support

           0       0.86      0.86      0.86       212
           1       0.90      0.90      0.90       295

    accuracy                           0.88       507
   macro avg       0.88      0.88      0.88       507
weighted avg       0.88      0.88      0.88       507



In [19]:
model_4 = SVC()
model_4.fit(X_train, y_train)
model_4.score(X_test, y_test)

0.9329388560157791

In [20]:
y_val_pred_4 = model_4.predict(X_test)
print(classification_report(y_test, y_val_pred_4))

              precision    recall  f1-score   support

           0       0.93      0.91      0.92       212
           1       0.93      0.95      0.94       295

    accuracy                           0.93       507
   macro avg       0.93      0.93      0.93       507
weighted avg       0.93      0.93      0.93       507



In [19]:
model_5 = RandomForestClassifier()
model_5.fit(X_train, y_train)
model_5.score(X_test, y_test)

0.9230769230769231

In [20]:
y_val_pred_5 = model_5.predict(X_test)
print(classification_report(y_test, y_val_pred_5))

              precision    recall  f1-score   support

           0       0.93      0.88      0.91       212
           1       0.92      0.96      0.94       295

    accuracy                           0.92       507
   macro avg       0.93      0.92      0.92       507
weighted avg       0.92      0.92      0.92       507



Best param from previous Hyperparameter

In [22]:
svm_grid = SVC(C=100, gamma=1)
svm_grid.fit(X_train, y_train)
svm_grid.score(X_test, y_test)

0.9467455621301775

In [23]:
svm_grid_y_pred = svm_grid.predict(X_test)
print(classification_report(y_test, svm_grid_y_pred))

              precision    recall  f1-score   support

           0       0.95      0.92      0.94       212
           1       0.95      0.96      0.95       295

    accuracy                           0.95       507
   macro avg       0.95      0.94      0.95       507
weighted avg       0.95      0.95      0.95       507



In [24]:
rdf_grid = RandomForestClassifier(n_estimators=400, min_samples_leaf=1, max_depth=90)
rdf_grid.fit(X_train, y_train)
rdf_grid.score(X_test, y_test)

0.9250493096646942

In [25]:
rdf_grid_y_pred = rdf_grid.predict(X_test)
print(classification_report(y_test, rdf_grid_y_pred))

              precision    recall  f1-score   support

           0       0.93      0.89      0.91       212
           1       0.92      0.95      0.94       295

    accuracy                           0.93       507
   macro avg       0.93      0.92      0.92       507
weighted avg       0.93      0.93      0.92       507



In [75]:
lgt_grid = LogisticRegression(C=1500, multi_class='ovr', solver='liblinear')
lgt_grid.fit(X_train, y_train)
lgt_grid.score(X_test, y_test)

0.9467455621301775

In [77]:
param_grid = {
 'C': [10, 100, 1000, 1500],
 'solver': ['liblinear'],  
 'multi_class': ['ovr']}
lgt_grid = GridSearchCV(LogisticRegression(), param_grid, refit = True, verbose = 3)
lgt_grid.fit(X_train, y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV 1/5] END C=10, multi_class=ovr, solver=liblinear;, score=0.932 total time=   0.7s
[CV 2/5] END C=10, multi_class=ovr, solver=liblinear;, score=0.924 total time=   0.9s
[CV 3/5] END C=10, multi_class=ovr, solver=liblinear;, score=0.934 total time=   0.7s
[CV 4/5] END C=10, multi_class=ovr, solver=liblinear;, score=0.922 total time=   0.5s
[CV 5/5] END C=10, multi_class=ovr, solver=liblinear;, score=0.936 total time=   0.5s
[CV 1/5] END C=100, multi_class=ovr, solver=liblinear;, score=0.941 total time=   0.7s
[CV 2/5] END C=100, multi_class=ovr, solver=liblinear;, score=0.939 total time=   0.6s
[CV 3/5] END C=100, multi_class=ovr, solver=liblinear;, score=0.937 total time=   0.6s
[CV 4/5] END C=100, multi_class=ovr, solver=liblinear;, score=0.927 total time=   0.7s
[CV 5/5] END C=100, multi_class=ovr, solver=liblinear;, score=0.948 total time=   0.7s
[CV 1/5] END C=1000, multi_class=ovr, solver=liblinear;, score=0.945 total 

GridSearchCV(estimator=LogisticRegression(),
             param_grid={'C': [10, 100, 1000, 1500], 'multi_class': ['ovr'],
                         'solver': ['liblinear']},
             verbose=3)

In [78]:
lgt_grid.best_estimator_

LogisticRegression(C=1500, multi_class='ovr', solver='liblinear')

In [21]:
model_2_best = LogisticRegression(C=1500, multi_class='ovr', solver='liblinear')
model_2_best.fit(X_train, y_train)
model_2_best.score(X_test, y_test)

0.9447731755424064

In [22]:
y_val_pred_2_best = model_2_best.predict(X_test)
print(classification_report(y_test, y_val_pred_2_best))

              precision    recall  f1-score   support

           0       0.94      0.93      0.93       212
           1       0.95      0.96      0.95       295

    accuracy                           0.94       507
   macro avg       0.94      0.94      0.94       507
weighted avg       0.94      0.94      0.94       507



In [23]:
import pickle
pickle.dump(model_2_best, open("best_logistic_model.pickle", "wb"))

In [24]:
loaded_model = pickle.load(open("best_logistic_model.pickle", "rb"))

In [25]:
y_val_pred_2_loaded_model = loaded_model.predict(X_test)
print(classification_report(y_test, y_val_pred_2_loaded_model))

              precision    recall  f1-score   support

           0       0.94      0.93      0.93       212
           1       0.95      0.96      0.95       295

    accuracy                           0.94       507
   macro avg       0.94      0.94      0.94       507
weighted avg       0.94      0.94      0.94       507



In [33]:
# save tfidf
import dill
dill.dump(tfidf_fit, open("tfidf_trans.dill", "wb"))

In [37]:
import numpy as np
x = tfidf_fit.transform(["น้ำมะนาวโซดารักษามะเร็งได้จริงๆ"])
pred_x = loaded_model.predict(x)
pred_x_2 = loaded_model.predict_proba(x)
print(pred_x)
print(pred_x_2)

[1]
[[7.12119948e-06 9.99992879e-01]]
